In [4]:
import pandas as pd
import numpy as np
df1=pd.read_csv('D1.csv')
test=pd.DataFrame()
print(df1.info())
print(df1.head(10))
df1.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1509 entries, 0 to 1508
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   patient_id  1509 non-null   int64  
 1   global_num  959 non-null    float64
 2   date        1509 non-null   object 
 3   location    1509 non-null   object 
 4   latitude    1509 non-null   float64
 5   longitude   1509 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 70.9+ KB
None
   patient_id  global_num        date               location   latitude  \
0  1000000001         2.0  22/01/2020   Gyeonggi-do_Gimpo-si  37.615246   
1  1000000001         2.0  24/01/2020          Seoul_Jung-gu  37.567241   
2  1000000002         5.0  26/01/2020     Seoul_Seongdong-gu  37.563992   
3  1000000002         5.0  27/01/2020    Seoul_Dongdaemun-gu  37.566262   
4  1000000002         5.0  28/01/2020       Seoul_Gangnam-gu  37.523674   
5  1000000004         7.0  30/01/2020      Seoul

patient_id      int64
global_num    float64
date           object
location       object
latitude      float64
longitude     float64
dtype: object

In [7]:
df1['date']=pd.to_datetime(df1['date'])

In [9]:
df1.dtypes

patient_id             int64
global_num           float64
date          datetime64[ns]
location              object
latitude             float64
longitude            float64
dtype: object

In [13]:
df2=df1[['patient_id','location']]

In [15]:
df2.info()
df2.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1509 entries, 0 to 1508
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   patient_id  1509 non-null   int64 
 1   location    1509 non-null   object
dtypes: int64(1), object(1)
memory usage: 23.7+ KB


,patient_id,location
0,1000000001,Gyeonggi-do_Gimpo-si
1,1000000001,Seoul_Jung-gu
2,1000000002,Seoul_Seongdong-gu
3,1000000002,Seoul_Dongdaemun-gu
4,1000000002,Seoul_Gangnam-gu
5,1000000004,Seoul_Jungnang-gu
6,1000000005,Seoul_Jungnang-gu
7,1000000006,Gyeonggi-do_Goyang-si
8,1000000007,Gyeonggi-do_Goyang-si
9,1000000008,Seoul_Jung-gu


In [16]:
route=df2.groupby(['location'])['patient_id'].apply(list)

In [34]:
route.head(10)

location
Busan_Buk-gu          [1100000105, 1100000108, 1100000129, 110000013...
Busan_Busanjin-gu     [1100000003, 1100000018, 1100000028, 110000002...
Busan_Dong-gu         [1100000002, 1100000071, 1100000075, 110000007...
Busan_Dongnae-gu      [1100000029, 1100000035, 1100000044, 110000004...
Busan_Gangseo-gu      [1100000094, 1100000105, 1100000108, 110000010...
Busan_Geumjeong-gu    [1100000014, 1100000049, 1100000057, 110000006...
Busan_Gijang-gun                   [1100000037, 1100000096, 1100000113]
Busan_Haeundae-gu     [1100000001, 1100000018, 1100000019, 110000002...
Busan_Jung-gu                      [1100000033, 6011000010, 6023000036]
Busan_Nam-gu          [1300000003, 1300000013, 1600000005, 160000001...
Name: patient_id, dtype: object

In [35]:
route1=df2.groupby(['patient_id'])['location'].apply(list)
route1.head(10)

patient_id
1000000001                [Gyeonggi-do_Gimpo-si, Seoul_Jung-gu]
1000000002    [Seoul_Seongdong-gu, Seoul_Dongdaemun-gu, Seou...
1000000004                                  [Seoul_Jungnang-gu]
1000000005                                  [Seoul_Jungnang-gu]
1000000006                              [Gyeonggi-do_Goyang-si]
1000000007                              [Gyeonggi-do_Goyang-si]
1000000008                                      [Seoul_Jung-gu]
1000000009                                  [Seoul_Jungnang-gu]
1000000010                                  [Seoul_Seongbuk-gu]
1000000011                       [Seoul_Jung-gu, Seoul_Mapo-gu]
Name: location, dtype: object

In [39]:
from apyori import apriori
from IPython.display import display
route1_list = list(route1)
results = list(apriori(route1_list, min_support = 0.0001, col_names=True))

display(route1_list[:5])

[['Gyeonggi-do_Gimpo-si', 'Seoul_Jung-gu'],
 ['Seoul_Seongdong-gu', 'Seoul_Dongdaemun-gu', 'Seoul_Gangnam-gu'],
 ['Seoul_Jungnang-gu'],
 ['Seoul_Jungnang-gu'],
 ['Gyeonggi-do_Goyang-si']]

In [37]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                         rule_set.support, rule.confidence, rule.lift]) 
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 
                                        'Confidence', 'Lift']) 

result_df = convert_apriori_results_to_pandas_df(results)

display(result_df.head(20))

,Left_side,Right_side,Support,Confidence,Lift
0,,Busan_Buk-gu,0.005612,0.005612,1.0
1,,Busan_Busanjin-gu,0.012346,0.012346,1.0
2,,Busan_Dong-gu,0.011223,0.011223,1.0
3,,Busan_Dongnae-gu,0.019080,0.019080,1.0
4,,Busan_Gangseo-gu,0.007856,0.007856,1.0
5,,Busan_Geumjeong-gu,0.006734,0.006734,1.0
6,,Busan_Gijang-gun,0.003367,0.003367,1.0
7,,Busan_Haeundae-gu,0.015713,0.015713,1.0
8,,Busan_Jung-gu,0.003367,0.003367,1.0
9,,Busan_Nam-gu,0.006734,0.006734,1.0


In [38]:
result_df = result_df.sort_values(by='Lift', ascending=False)
display(result_df.head(10))

,Left_side,Right_side,Support,Confidence,Lift
6153,"Gyeonggi-do_Icheon-si,Seoul_Yongsan-gu","Seoul_Yeongdeungpo-gu,Seoul_Mapo-gu",0.001122,1.0,891.0
6355,"Busan_Dong-gu,Chungcheongbuk-do_Goesan-gun,Chu...",Busan_Yeongdo-gu,0.001122,1.0,891.0
6352,"Chungcheongbuk-do_Goesan-gun,Chungcheongbuk-do...","Busan_Dong-gu,Busan_Nam-gu",0.001122,1.0,891.0
6351,"Chungcheongbuk-do_Goesan-gun,Chungcheongbuk-do...","Busan_Dong-gu,Busan_Yeongdo-gu",0.001122,1.0,891.0
6350,"Chungcheongbuk-do_Goesan-gun,Busan_Yeongdo-gu,...","Busan_Dong-gu,Chungcheongbuk-do_Cheongju-si",0.001122,1.0,891.0
6349,"Busan_Yeongdo-gu,Chungcheongbuk-do_Cheongju-si...","Busan_Dong-gu,Chungcheongbuk-do_Goesan-gun",0.001122,1.0,891.0
6348,"Busan_Dong-gu,Chungcheongbuk-do_Cheongju-si,Ch...","Busan_Nam-gu,Busan_Yeongdo-gu",0.001122,1.0,891.0
6347,"Busan_Dong-gu,Chungcheongbuk-do_Goesan-gun,Bus...","Chungcheongbuk-do_Cheongju-si,Busan_Nam-gu",0.001122,1.0,891.0
6346,"Busan_Dong-gu,Chungcheongbuk-do_Cheongju-si,Bu...","Busan_Nam-gu,Chungcheongbuk-do_Goesan-gun",0.001122,1.0,891.0
6345,"Busan_Dong-gu,Chungcheongbuk-do_Goesan-gun,Bus...","Chungcheongbuk-do_Cheongju-si,Busan_Yeongdo-gu",0.001122,1.0,891.0


In [41]:
result_df = result_df.sort_values(by='Lift', ascending=False)
print(result_df.head(10))

                                              Left_side  \
6153             Gyeonggi-do_Icheon-si,Seoul_Yongsan-gu   
6340     Chungcheongbuk-do_Cheongju-si,Busan_Yeongdo-gu   
6351  Chungcheongbuk-do_Goesan-gun,Chungcheongbuk-do...   
6350  Chungcheongbuk-do_Goesan-gun,Busan_Yeongdo-gu,...   
6349  Busan_Yeongdo-gu,Chungcheongbuk-do_Cheongju-si...   
6348  Busan_Dong-gu,Chungcheongbuk-do_Cheongju-si,Ch...   
6347  Busan_Dong-gu,Chungcheongbuk-do_Goesan-gun,Bus...   
6346  Busan_Dong-gu,Chungcheongbuk-do_Cheongju-si,Bu...   
6345  Busan_Dong-gu,Chungcheongbuk-do_Goesan-gun,Bus...   
6344  Busan_Dong-gu,Chungcheongbuk-do_Cheongju-si,Bu...   

                                             Right_side   Support  Confidence  \
6153                Seoul_Yeongdeungpo-gu,Seoul_Mapo-gu  0.001122         1.0   
6340  Busan_Nam-gu,Busan_Dong-gu,Chungcheongbuk-do_G...  0.001122         1.0   
6351                     Busan_Dong-gu,Busan_Yeongdo-gu  0.001122         1.0   
6350        Busan_Dong-gu,